# Summary:
- Imbalanced sampling (SMOTE) is applied.
- 1D Cov + two layers of LSTM is tested.


In [1]:
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE, ADASYN

import numpy as np
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
np.random.seed(1)

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


# 1. Data Preparation

In [2]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [3]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [4]:
# Use SMOTE oversampling to generate more training data
X_resampled, y_resampled = SMOTE().fit_sample(vectors, data_Y)

In [5]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(X_resampled, y_resampled, test_size=0.25)

In [6]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [7]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((12883, 1024), (12883,), (4295, 1024), (4295,), 10216)

In [8]:
xtr, xt = X_train_array.shape, X_test_array.shape

In [9]:
# reshape dataset
X_train = X_train_array.reshape(xtr[0],32,32) 
Y_train = np.eye(2)[Y_train_array.astype(int)] 
X_test = X_test_array.reshape(xt[0],32,32) 
Y_test = np.eye(2)[Y_test_array.astype(int)] 

# 2. LSTM

In [10]:
def model(input_shape):
    
    X_input = Input(shape = input_shape)
    
    # Cov layer
    X = Conv1D(filters=768, kernel_size=15, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.6)(X)                                 # dropout
    
    # LSTM layer 1
    X = LSTM(512, return_sequences=True)(X)
    X = Dropout(0.5)(X)
    
    # LSTM layer 2 
    X = LSTM(512)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)  
    # Softmax layer
    X = Activation('softmax')(X)
    
    # Create Model instance
    model = Model(inputs = X_input, outputs = X)
        
    return model

In [11]:
model = model(input_shape = (32,32))

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 768)            369408    
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 768)            3072      
_________________________________________________________________
activation_1 (Activation)    (None, 5, 768)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 768)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 512)            2623488   
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 512)            0         
__________

In [13]:
opt = Adam(lr=0.003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [14]:
model.fit(X_train, Y_train, batch_size = 64, epochs=100)

Epoch 1/100
12883/12883 [==============================] - 7s 561us/step - loss: 0.4807 - acc: 0.7815
Epoch 2/100
12883/12883 [==============================] - 5s 422us/step - loss: 0.3439 - acc: 0.8602
Epoch 3/100
12883/12883 [==============================] - 5s 424us/step - loss: 0.2628 - acc: 0.8978
Epoch 4/100
12883/12883 [==============================] - 5s 422us/step - loss: 0.2068 - acc: 0.9207
Epoch 5/100
12883/12883 [==============================] - 5s 419us/step - loss: 0.1611 - acc: 0.9389
Epoch 6/100
12883/12883 [==============================] - 5s 422us/step - loss: 0.1274 - acc: 0.9535
Epoch 7/100
12883/12883 [==============================] - 5s 422us/step - loss: 0.0961 - acc: 0.9661
Epoch 8/100
12883/12883 [==============================] - 5s 423us/step - loss: 0.0822 - acc: 0.9719
Epoch 9/100
12883/12883 [==============================] - 5s 423us/step - loss: 0.0698 - acc: 0.9759
Epoch 10/100
12883/12883 [==============================] - 5s 425us/step - loss: 

12883/12883 [==============================] - 5s 409us/step - loss: 0.0049 - acc: 0.9983
Epoch 79/100
12883/12883 [==============================] - 5s 407us/step - loss: 0.0060 - acc: 0.9974
Epoch 80/100
12883/12883 [==============================] - 5s 409us/step - loss: 0.0067 - acc: 0.9977
Epoch 81/100
12883/12883 [==============================] - 5s 409us/step - loss: 0.0063 - acc: 0.9976
Epoch 82/100
12883/12883 [==============================] - 5s 409us/step - loss: 0.0067 - acc: 0.9974
Epoch 83/100
12883/12883 [==============================] - 5s 410us/step - loss: 0.0070 - acc: 0.9977
Epoch 84/100
12883/12883 [==============================] - 5s 409us/step - loss: 0.0041 - acc: 0.9988
Epoch 85/100
12883/12883 [==============================] - 5s 409us/step - loss: 0.0049 - acc: 0.9980
Epoch 86/100
12883/12883 [==============================] - 5s 409us/step - loss: 0.0039 - acc: 0.9987
Epoch 87/100
12883/12883 [==============================] - 5s 409us/step - loss: 0.00

In [15]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4295/4295 [==============================] - 1s 186us/step
Validation set accuracy =  0.9336437718277066
